# Quiz 10

"""
Your task in this exercise has two steps:

- audit the OSMFILE and change the variable 'mapping' to reflect the changes needed to fix 
    the unexpected street types to the appropriate ones in the expected list.
    You have to add mappings only for the actual problems you find in this OSMFILE,
    not a generalized solution, since that may and will depend on the particular area you are auditing.
- write the update_name function, to actually fix the street name.
    The function takes a string with street name as an argument and should return the fixed name
    We have provided a simple test so that you see what exactly is expected
"""

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

In [2]:
OSMFILE = "example.osm.xml"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

In [3]:
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

In [4]:
# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue"
            }

In [5]:
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

In [6]:
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types


In [7]:
def update_name(name, mapping):
    k = mapping.keys()
    key_list = list(k)
    for abrev in key_list:
        if abrev in name.split():
            name = name.replace(abrev,mapping[abrev])

    return name

In [8]:
def test():
    st_types = audit(OSMFILE)
    assert len(st_types) == 3
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"

In [9]:
if __name__ == '__main__':
    test()

{'Ave': {'North Lincoln Ave', 'N. Lincoln Ave'},
 'Rd.': {'Baldwin Rd.'},
 'St.': {'West Lexington St.'}}
North Lincoln Ave => North Lincoln Avenue
N. Lincoln Ave => N. Lincoln Avenue
Baldwin Rd. => Baldwin Road
West Lexington St. => West Lexington Street


In [10]:
st_types = audit(OSMFILE)
st_types

defaultdict(set,
            {'Ave': {'N. Lincoln Ave', 'North Lincoln Ave'},
             'Rd.': {'Baldwin Rd.'},
             'St.': {'West Lexington St.'}})

In [11]:
for st_type, ways in st_types.items():  # python 3 uses d.items() in place of d.iteritems()
    print("key: {}, value: {}".format(st_type, ways))
    for name in ways:
        print('Names are : ',name)

key: Ave, value: {'North Lincoln Ave', 'N. Lincoln Ave'}
Names are :  North Lincoln Ave
Names are :  N. Lincoln Ave
key: Rd., value: {'Baldwin Rd.'}
Names are :  Baldwin Rd.
key: St., value: {'West Lexington St.'}
Names are :  West Lexington St.


In [12]:
mapping

{'Ave': 'Avenue', 'Rd.': 'Road', 'St': 'Street', 'St.': 'Street'}

In [13]:
k = mapping.keys()
t = list(k)
t

['St', 'St.', 'Rd.', 'Ave']

In [14]:
nm = 'Baldwin Rd.'
nm.split()

['Baldwin', 'Rd.']

In [15]:
nm

'Baldwin Rd.'

In [16]:
'Rd.' in nm.split()

True

In [17]:
'Rd.' in 'Baldwin Rd.'

True

In [18]:
mapping['Ave']

'Avenue'

In [19]:
update_name('West Lexington St.',mapping)

'West Lexington Street'